# Problem Solving


## 1. 

The practice of introspection would allow the programmer to examine the human cognitive process, studying its working, reasoning and assumptions that allow it to solve problems. Following a train of reasoning is an integral part of translating our sometimes intuitive methods of soving problems that don't require intentional thought and may be missed.  Our brains can fill in missing information or extrapolate from incomplete data without our conscious thought so that we can be efficient. These shortcuts the brain takes makes problem solving easy for us, but a computer model without these shortcuts intentionally programmed would not be able to solve the same problems. 

While it may provide a path to solve problems, it would be hard to closely implement a solution similar to that of the mind because the mind has learned the world and created a schema that may be unique to each person and cannot be examined even with introspection. With this uniqueness comes biases that may be ignored or passed into the model. Introspection would be a good way to model problem solving, but there are some parts of cognition that will be missed even with intense reflection.   
  

# 2. 


## Travelling Salesman Problem

Each state is a list of the cities and different states have the cities in different orders. The action is swapping any two cities with each other to make a new state that is evaluated. 

Simulated annealing usually does better than hill-climbing. As, hill-climbing calculates its route, it would not go down a path with a larger distance even if total distance is smaller while the random starting locations of simulated annealing would allow it to choose those routes that are shortest. 



In [17]:
"""Implementation of the Travelling Salesman Problem for 
@author: James Eapen (jpe4)
@date: 2020 Feb 27
"""


import sys
import random
sys.path.append("/home/james/Documents/Calvin/CS-344/cs344-code/tools/aima")

from search import Problem, hill_climbing, simulated_annealing, \
    exp_schedule, genetic_search
import logging
import time

class TSP(Problem):
    def __init__(self, initial, distances):
        self.initial = initial
        self.distances = distances

    def actions(self, state):
        """select two cities at random to swap and pass a list of their indices to actions"""

        swap_list = random.sample(self.initial, 2)
        index1, index2 = self.initial.index(swap_list[0]), self.initial.index(swap_list[1])
        actions = [[index1, index2]]
        return actions

    def result(self, state, action):
        """make a copy of the previous state and swap the two cities;
         add the first city of the new state to the end for a round trip"""

        new_state = self.initial[:]
        new_state[action[0]], new_state[action[1]] = new_state[action[1]], new_state[action[0]]
        new_state.append(new_state[0])
        return new_state

    def value(self, state):
        """find the total round trip distance of travel between the cities in the state"""
        
        total_dist = 0
        for i in range(len(state)-1):
            city1 = state[i]
            city2 = state[i+1]

            if city1 > city2:
                total_dist += self.distances[(city2, city1)]
            else:
                total_dist += self.distances[(city1, city2)]

        return -total_dist
                       

cities = ["A", "B", "C", "D"]
distances = {("A", "B"): 1, ("A", "C"): 2, ("A", "D"): 4,
                ("B", "C"): 3.5, ("B", "D"): 5, 
                ("C", "D"): 3}
        
tsp = TSP(cities, distances)

hill_climbing_test = hill_climbing(tsp)

print("Hill Climbing")
print("Solution: " + str(hill_climbing_test))
print("Value: " + str(tsp.value(hill_climbing_test)))
print("\n")

simulated_annealing_test = simulated_annealing(tsp)
print("Simulated Annealing")
print("Solution: " + str(simulated_annealing_test))
print("Value: " + str(tsp.value(simulated_annealing_test)))
        

Hill Climbing
Solution: ['A', 'B', 'C', 'D']
Value: -7.5


Simulated Annealing
Solution: ['A', 'B', 'D', 'C', 'A']
Value: -11


# 3.

## Class Scheduler

This implementation makes each class a variable and neighbor of every other class since the time of each class interacts with every other class. The domains have every possible combination of class, time, professor and location and the constraint checker makes sure they follow the constraints.  
There are also class assignments to each professor and the constraint checker makes sure that the schedule has each professor scheduled only for their assigned class.

In [2]:
"""CS Class Scheduling
@author: James Eapen (jpe4)
@date: 2020 Feb 27
"""

import sys
sys.path.append("/home/james/Documents/Calvin/CS-344/cs344-code/tools/aima")

from csp import CSP, min_conflicts, backtracking_search, AC3, parse_neighbors
import random

def Scheduler():
    courses = ['cs108', 'cs112', 'cs212', 'cs232', 'cs344', 'cs342']
    professors = ['hplantin', 'adams', 'vnorman', 'kvlinden', 'pbailey']
    times = ['mwf900', 'tth1030', 'mwf1130', 'mwf1230', 't1800', 'mwf1430']
    classrooms = ['sb382', 'nh253']
    assignments = {'cs108': 'kvlinden',
                    'cs112': 'adams',
                    'cs212': 'hplantin',
                    'cs232': 'vnorman',
                    'cs342': 'pbailey',
                    'cs344': 'kvlinden'}

    variables = courses 
    domains = {}
    # make all possible combinations with class as key
    for course in variables:
        domains[course]= []
        for time in times:
            for classroom in classrooms:
                for professor in professors:
                    domains[course].append((professor, time, classroom))

    # set all classes neighbors of each other because each affects every other
    neighbors = parse_neighbors('''cs108: cs112 cs212 cs344 cs232 cs342; 
                            cs112: cs212 cs232 cs344 cs342; 
                            cs212: cs232 cs344 cs342; 
                            cs232: cs344 cs342; 
                            cs344: cs342''', variables)
    
    # set constraints
    def scheduling_constraints(A, a, B, b):
        # if same prof
        if a[0] == b[0]:

            # and same time
            if a[1] == b[1]:
                return False
            # or same room 
            if a[2] == b[2]:
                return False
        
        # if same time
        if a[1] == b[1]:

            # and same prof
            if a[0] == b[0]:
                return False
            # or same room
            if a[2] == b[0]:
                return False

        # if same room
        if a[2] == b[2]:

            # and same prof
            if a[0] == b[0]:
                return False

            # or same time
            if a[1] == b[1]:
                return False 

        # check if class has correctly assigned prof
        if (assignments[A] != a[0]) or (assignments[B] != b[0]):
            return False

        return True
    return CSP(variables, domains, neighbors, scheduling_constraints)

p = Scheduler()
result = backtracking_search(p)

for each_class in result:
    print("Class: \t", each_class)
    print("Prof: \t", result[each_class][0])
    print("Room: \t", result[each_class][2])
    print("Time: \t", result[each_class][1])
    print()

Class: 	 cs108
Prof: 	 kvlinden
Room: 	 sb382
Time: 	 mwf900

Class: 	 cs112
Prof: 	 adams
Room: 	 nh253
Time: 	 mwf900

Class: 	 cs212
Prof: 	 hplantin
Room: 	 sb382
Time: 	 tth1030

Class: 	 cs232
Prof: 	 vnorman
Room: 	 nh253
Time: 	 tth1030

Class: 	 cs344
Prof: 	 kvlinden
Room: 	 nh253
Time: 	 mwf1130

Class: 	 cs342
Prof: 	 pbailey
Room: 	 sb382
Time: 	 mwf1130

